In [2]:
#!pip install scikit-plot

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
)

In [52]:
df_train = pd.read_csv('train_nolog.csv')

In [53]:
# X = df_train.values
# y = np.array(df_train['titleType'])

In [54]:
del df_train['originalTitle']
del df_train['rating']
del df_train['genres']
del df_train['countryOfOrigin']
del df_train['isAdult']
del df_train['canHaveEpisodes']

## Partitioning

In [55]:
df_train.head()

,startYear,runtimeMinutes,awardWins,numVotes,totalImages,totalVideos,totalCredits,titleType,AwNmExWins,numRegions,ratingMean,ReviewsTotal
0,1995,40,0,779,1,0,21,tvEpisode,0,1,7.5,1
1,1982,28,0,11,1,0,1,videoGame,0,1,5.5,0
2,1992,92,0,38,1,0,24,movie,0,1,5.5,1
3,2021,53,0,15,22,0,56,tvSeries,0,1,5.5,0
4,2020,80,1,15718,347,2,47,tvSeries,1,32,7.5,456


In [30]:
from sklearn.model_selection import train_test_split

In [31]:
X = df_train.drop(columns=['titleType'])
y = df_train['titleType']

In [33]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, stratify=y, random_state=42
)
# with stratify
print(np.unique(y, return_counts=True)[1] / len(y))
print(np.unique(y_train, return_counts=True)[1] / len(y_train))
print(np.unique(y_test, return_counts=True)[1] / len(y_test))

[0.33262166 0.15259027 0.29312402 0.01312402 0.05293564 0.08200942
 0.00251177 0.00929356 0.04803768 0.01375196]
[0.33260073 0.15259027 0.29314495 0.01308216 0.05295657 0.08205128
 0.00251177 0.0093145  0.04803768 0.0137101 ]
[0.33265306 0.15259027 0.29309262 0.01318681 0.05290424 0.08194662
 0.00251177 0.00926217 0.04803768 0.01381476]


### Normalization

In [34]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [35]:
norm = MinMaxScaler()
norm.fit(X_train)

X_train_norm = norm.transform(X_train)
X_test_norm = norm.transform(X_test)

## Nayve Bayesian

In [36]:
from sklearn.naive_bayes import GaussianNB, CategoricalNB

In [37]:
clf = GaussianNB()

In [38]:
%%time
clf.fit(X_train, y_train)

CPU times: total: 15.6 ms
Wall time: 14.4 ms


GaussianNB()

In [39]:
y_pred = clf.predict(X_test)
y_pred

array(['video', 'video', 'video', ..., 'tvEpisode', 'tvEpisode', 'short'],
      dtype='<U12')

In [42]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       movie       0.88      0.17      0.28      2119
       short       0.82      0.64      0.72       972
   tvEpisode       0.66      0.86      0.74      1867
tvMiniSeries       0.12      0.14      0.13        84
     tvMovie       0.08      0.27      0.13       337
    tvSeries       0.52      0.12      0.19       522
     tvShort       0.04      0.62      0.07        16
   tvSpecial       0.01      0.02      0.01        59
       video       0.17      0.57      0.26       306
   videoGame       0.36      0.06      0.10        88

    accuracy                           0.46      6370
   macro avg       0.36      0.35      0.26      6370
weighted avg       0.67      0.46      0.46      6370



In [ ]:
clf.predict_proba(X_test)

array([[3.05055218e-156, 1.98469602e-002, 9.80153040e-001],
       [2.50142195e-277, 3.75932821e-012, 1.00000000e+000],
       [1.00000000e+000, 2.74048199e-017, 2.82825801e-026],
       [3.77047257e-248, 6.87918788e-010, 9.99999999e-001],
       [1.00000000e+000, 1.81924966e-016, 2.52343415e-026],
       [1.00000000e+000, 5.84024273e-017, 8.14494337e-027],
       [1.00000000e+000, 1.12344252e-016, 5.92512274e-026],
       [6.19173845e-176, 4.67941390e-004, 9.99532059e-001],
       [1.10238158e-032, 9.99999857e-001, 1.43256972e-007],
       [7.56032487e-075, 9.99968118e-001, 3.18821921e-005],
       [1.00000000e+000, 1.66546269e-010, 2.54675324e-019],
       [3.73265040e-154, 6.58305961e-002, 9.34169404e-001],
       [0.00000000e+000, 1.70481203e-012, 1.00000000e+000],
       [1.00000000e+000, 6.04531187e-012, 4.37830158e-020],
       [1.35690609e-092, 9.96128400e-001, 3.87159969e-003],
       [7.97250163e-267, 3.29344440e-010, 1.00000000e+000],
       [1.86513942e-098, 9.78842746e-001

In [47]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
plot_roc(y_test, clf.predict_proba(X_test))
plt.show()
print(roc_auc_score(y_test, clf.predict_proba(X_test), multi_class="ovr", average="macro"))

NameError: name 'plot_roc' is not defined

In [59]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.naive_bayes import GaussianNB

label_encoder = LabelEncoder()

# Binarizza le classi (necessario per ROC Curve multiclasse)
y_test_bin = label_binarize(y_test, classes=range(len(label_encoder.classes_)))
n_classes = y_test_bin.shape[1]

# Ottieni le probabilità predette
y_score = model.predict_proba(X_test.toarray())

# Calcolo della ROC Curve e dell'AUC per ciascuna classe
fpr = {}
tpr = {}
roc_auc = {}

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Tracciamento delle ROC curve
plt.figure(figsize=(10, 7))
colors = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black']
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], color=colors[i % len(colors)],
             lw=2, label=f'ROC curve (class {label_encoder.inverse_transform([i])[0]}) (area = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)  # Linea diagonale
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid()
plt.show()

AttributeError: 'LabelEncoder' object has no attribute 'classes_'